In [1]:
import os
from dotenv import load_dotenv  # Load environment variables from a .env file
load_dotenv()  # Load the environment variables

True

In [2]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY","")  # Retrieve the OpenAI API key from the environment variables
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY","")  # Retrieve the LangChain API key from the environment variables
os.environ["LANGCHAIN_TRACING_V2"] = "true"  # Enable LangChain tracing for debugging and monitoring
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT","")  # Set the LangChain project name for organization

In [3]:
## import groq
from langchain_groq import ChatGroq  # Import the GroqChat model for use in LangChain
llm = ChatGroq(model="openai/gpt-oss-120b")  # Initialize the GroqChat model with specific parameters
print(llm)  # Print the model details to verify initialization

client=<groq.resources.chat.completions.Completions object at 0x0000014F5287B3A0> async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000014F528B1810> model_name='openai/gpt-oss-120b' model_kwargs={} groq_api_key=SecretStr('**********')


In [4]:
from langchain_core.messages import HumanMessage  # Import HumanMessage for creating messages in LangChain

llm.invoke([
    HumanMessage(
        content="What is the capital of France?"
    )]
)  # Invoke the model with a human message asking for the capital of France

AIMessage(content='The capital of France is **Paris**.', additional_kwargs={'reasoning_content': 'The user asks: "What is the capital of France?" This is a straightforward factual question. The answer is Paris. Provide answer.'}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 78, 'total_tokens': 124, 'completion_time': 0.083180669, 'prompt_time': 0.004455878, 'queue_time': 0.049660902, 'total_time': 0.087636547}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_a09bde29de', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--73de0e6e-6630-4116-a2da-da95d84569c3-0', usage_metadata={'input_tokens': 78, 'output_tokens': 46, 'total_tokens': 124})

In [5]:
llm.invoke("what is the capital of France?")  # Invoke the model with a string input directly

AIMessage(content='The capital of France is **Paris**.', additional_kwargs={'reasoning_content': 'The user asks: "what is the capital of France?" Straightforward answer: Paris. Provide answer.'}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 78, 'total_tokens': 118, 'completion_time': 0.073112541, 'prompt_time': 0.004566683, 'queue_time': 0.043770968, 'total_time': 0.077679224}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_4a19b1544c', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--3d8a1e87-3177-4f14-8d6e-ad93e92afb66-0', usage_metadata={'input_tokens': 78, 'output_tokens': 40, 'total_tokens': 118})

In [8]:
#lets check if the model is able to remember the previous question or messaes
from langchain_core.messages import HumanMessage, AIMessage  # Import HumanMessage and AIMessage for creating messages in LangChain
llm.invoke([
    HumanMessage(content="Hey my name is John"),
    AIMessage(content = "Hi John, how can I help you?"),
    HumanMessage(content="hey do you know my name?")
])  # Invoke the model with a sequence of   messages to check if it remembers the context

AIMessage(content='Yes— you told me your name is **John**. How can I help you today?', additional_kwargs={'reasoning_content': 'The user asks "hey do you know my name?" The assistant already knows from previous message that the user said "Hey my name is John". So answer: yes, you know it\'s John. Also ask how can help.'}, response_metadata={'token_usage': {'completion_tokens': 74, 'prompt_tokens': 102, 'total_tokens': 176, 'completion_time': 0.135052987, 'prompt_time': 0.005935127, 'queue_time': 0.192191353, 'total_time': 0.140988114}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_213abb2467', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--8d9b66b2-7b74-4ffb-9501-f3cc846439a0-0', usage_metadata={'input_tokens': 102, 'output_tokens': 74, 'total_tokens': 176})

In [21]:
#we will create a chatbot with messae history
from langchain_core.messages import HumanMessage, AIMessage  # Import HumanMessage and AIMessage for creating messages in LangChain
from langchain_community.chat_message_histories import ChatMessageHistory  # Import InMemoryChatMessageHistory for storing chat messages in memory
from langchain_core.chat_history import  BaseChatMessageHistory  # Import the base class for chat message history
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnablePassthrough  # Import RunnablePassthrough for passing through messages



store={}


def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if(session_id not in store):
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history= RunnableWithMessageHistory(llm,get_session_history)

In [35]:

with_message_history.invoke(
                            [
    HumanMessage(content="what is my name?"),
    
    ],config={
    "configurable":{"session_id": "12345"}  # Unique identifier for the session
    
}
    )  # Invoke the model with a sequence of messages and session configuration

AIMessage(content='Your name is John.', additional_kwargs={'reasoning_content': 'The user asks "what is my name?" The conversation earlier: user said "Hey my name is John". So answer: John. Should be short.'}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 325, 'total_tokens': 371, 'completion_time': 0.083236358, 'prompt_time': 0.013803132, 'queue_time': 0.045547448, 'total_time': 0.09703949}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_82669fd91d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--89be80c9-f006-4187-be69-0fb1b98464e2-0', usage_metadata={'input_tokens': 325, 'output_tokens': 46, 'total_tokens': 371})

In [36]:

with_message_history.invoke(
                            [
    HumanMessage(content="what is my name?"),
    
    ],config={
    "configurable":{"session_id": "1245"}  # Unique identifier for the session
    
}
    )  # Invoke the model with a sequence of messages and session configuration

AIMessage(content='I don’t have any information about your name. If you’d like me to address you by a particular name, just let me know!', additional_kwargs={'reasoning_content': 'The user asks "what is my name?" We have no prior context. According to policy, we cannot infer or guess. Should respond that I don\'t know. Provide suggestion to tell us.'}, response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 119, 'total_tokens': 195, 'completion_time': 0.141600863, 'prompt_time': 0.005936807, 'queue_time': 0.044875554, 'total_time': 0.14753767}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_82669fd91d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--7bb866dc-c64d-4397-b20c-62552d5d8df0-0', usage_metadata={'input_tokens': 119, 'output_tokens': 76, 'total_tokens': 195})

In [37]:
with_message_history.invoke(
                            [
    HumanMessage(content="my name is mari"),
    
    ],config={
    "configurable":{"session_id": "1245"}  # Unique identifier for the session
    
}
    ) 



AIMessage(content='Got it—your name is **Mari**. If you’d like me to use a different name or spelling, just let me know!', additional_kwargs={'reasoning_content': 'The user says "my name is mari". The user is now providing their name. The assistant should respond acknowledging. Should use the name "Mari" (capitalized). The user wants to know what is my name? The user says "my name is mari". The assistant should respond with "Your name is Mari". Should be polite.'}, response_metadata={'token_usage': {'completion_tokens': 106, 'prompt_tokens': 161, 'total_tokens': 267, 'completion_time': 0.193463214, 'prompt_time': 0.007595733, 'queue_time': 0.043193734, 'total_time': 0.201058947}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_4a19b1544c', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--952e9beb-7b5a-4125-aa0c-b5ea500a0411-0', usage_metadata={'input_tokens': 161, 'output_tokens': 106, 'total_tokens': 267})

In [38]:
with_message_history.invoke(
                            [
    HumanMessage(content="what  is my name?"),
    
    ],config={
    "configurable":{"session_id": "1245"}  # Unique identifier for the session
    
}
    ) 


AIMessage(content='You’ve told me your name is **Mari**. If you’d like me to use a different name or spelling, just let me know!', additional_kwargs={'reasoning_content': 'The user asks again "what is my name?" The user previously said "my name is mari". So we should answer "Your name is Mari." Also maybe ask if they want anything else.'}, response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 205, 'total_tokens': 283, 'completion_time': 0.140705228, 'prompt_time': 0.008974085, 'queue_time': 0.045720492, 'total_time': 0.149679313}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_a09bde29de', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--43ffc152-ecb5-4eef-b37b-74f2ca455a1c-0', usage_metadata={'input_tokens': 205, 'output_tokens': 78, 'total_tokens': 283})